In [1]:
# Reference: https://medium.com/radon-dev/item-item-collaborative-filtering-with-binary-or-unary-data-e8f0b465b2c3

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

dtypes={
    'request': str
}
data = pd.read_csv('user_activity_raw.csv')

/Users/celena/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5,6,7,8,15,16,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data.columns

Index(['request', 'user_id', 'created_at', 'max_retries', 'document_id', 'id',
       'file_id', 'client_updated_at', 'error', 'google_error', 'page',
       'per_page', 'q', 'alert_for', 'alert_type', 'comment', 'date',
       'user_id.1', 'doc_type', 'group_id', 'text', 'directory_user_id'],
      dtype='object')

In [9]:
user_doc_interaction = data.dropna(subset=['document_id'])

In [10]:
user_doc_interaction['request'].unique()

array(['shares#create', 'comments#index', 'comments#create',
       'likes#toggle', 'likes#index'], dtype=object)

In [11]:
user_doc_interaction = user_doc_interaction.iloc[:,0:5].drop(columns=['request', 'created_at', 'max_retries'])
user_doc_interaction.head()

,user_id,document_id
8666,2,19900145-8aee-4fec-9352-bf1f294e6ef0
10021,3,5dd2d65b-3f87-45cf-a517-6a18af7393e6
10022,3,5dd2d65b-3f87-45cf-a517-6a18af7393e6
10024,3,31155858-4856-425a-9d33-ec1bfe1bef87
10025,3,31155858-4856-425a-9d33-ec1bfe1bef87


In [15]:
grouped_by_id = user_doc_interaction.groupby(['user_id'])
grouped_by_id.head()

,user_id,document_id
8666,2,19900145-8aee-4fec-9352-bf1f294e6ef0
10021,3,5dd2d65b-3f87-45cf-a517-6a18af7393e6
10022,3,5dd2d65b-3f87-45cf-a517-6a18af7393e6
10024,3,31155858-4856-425a-9d33-ec1bfe1bef87
10025,3,31155858-4856-425a-9d33-ec1bfe1bef87
10083,3,87122f87-99fe-473e-8547-f6b5ca1892c1
48046,10,336940
48047,10,336940
48792,10,320143
49141,10,358007
